In [1]:
import pandas as pd
import string
import textblob
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
import re

In [2]:
df = pd.read_csv("train.csv")
text = list(df["text"])
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


### Preprocessing Functions

#### Remove website links

In [3]:
def remove_links(text):
    link_pattern = re.compile(r'(https?:\/\/(t.co)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6})\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
    return link_pattern.sub(r'', text)

#### Remove punctuation

In [4]:
def remove_punctuation(text):
    punctuation_pattern = re.compile(r"[\!\$\%\&\(\)\*\+,-./:;<=>\?\[\\\]\^\_\`\{\|\}~]*")
    return punctuation_pattern.sub(r'', text)

#### Remove emojis

In [5]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

#### Remove @ mentions

In [6]:
def remove_mentions(text):
    mentions_pattern = re.compile(r'^@\w*')
    return mentions_pattern.sub(r'', text)

In [7]:
#nltk.download() Opens a GUI to download the stopword corpus

In [10]:
stopwords = nltk.corpus.stopwords.words("English")

tokenizer = TweetTokenizer()
tokens = [tokenizer.tokenize(tweet) for tweet in text]
tokens = [[token.strip() for token in tweets] for tweets in tokens]
filtered_tokens = [[token for token in tweets if token.lower() not in stopwords] for tweets in tokens]
filtered_tweets = [' '.join(x) for x in filtered_tokens]


filtered_tweets = [remove_links(tweet) for tweet in filtered_tweets]
#filtered_tweets = [remove_punctuation(tweet) for tweet in filtered_tweets]
filtered_tweets = [remove_emoji(tweet) for tweet in filtered_tweets] 
#filtered_tweets = [remove_mentions(tweet) for tweet in filtered_tweets] 

df['text'] = filtered_tweets

### Find all hashtags in the corpus

In [11]:
pattern = r'\B(\#[a-zA-Z]+\b)(?!;)'
hashtags = [re.findall(pattern, x) for x in filtered_tweets]
df['Hashtags'] = hashtags
df

,id,keyword,location,text,target,Hashtags
0,1,NaN,NaN,Deeds Reason #earthquake May ALLAH Forgive us,1,[#earthquake]
1,4,NaN,NaN,Forest fire near La Ronge Sask . Canada,1,[]
2,5,NaN,NaN,residents asked ' shelter place ' notified off...,1,[]
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,[#wildfires]
4,7,NaN,NaN,got sent photo Ruby #Alaska smoke #wildfires p...,1,"[#Alaska, #wildfires]"
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding bridge collapse nearb...,1,[]
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest control wild fires C...,1,[]
7610,10871,NaN,NaN,M1 . 94 [ 01:04 UTC ] ? 5km Volcano Hawaii .,1,[]
7611,10872,NaN,NaN,Police investigating e-bike collided car Littl...,1,[]


In [12]:
df.to_csv("train-cleaned", index=False)